In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [6]:
afi_base_data = pd.read_excel('AFI data/AFI data_coductivity and milk yield_basedata.xlsx')

In [52]:
afi_base_data.head()
# in the below dataset, I need only the columns, 
# what i want see, for each ID, populate 30days before the minimum date and populate 30 days after the maximum date.

,ID,Grp.,Lact.,Parity,dim,Lactstage,Avg.Conductivity,DailyYield,Daily avge yield,Daily yield (%),day,Trt,Batch,Bacteria,Gram,Type,baccure,date
0,893,4,4,4,84,E,9.5,80.080617,.,NaN,-7,CON,1,SU,P,E,1.0,2023-12-07
1,919,4,4,4,44,E,9.9,66.31497,.,NaN,-7,CON,1,MBG,P,C,1.0,2023-12-07
2,971,11,1,1,365,L,0,0,.,NaN,-7,TRT,1,SC,P,E,1.0,2023-12-07
3,1643,4,3,3,86,E,10.2,47.511766,.,NaN,-7,TRT,1,MBG,P,C,1.0,2023-12-07
4,1668,4,3,3,77,E,10,101.61314,.,NaN,-7,CON,1,SA,P,C,1.0,2023-12-07


In [51]:
afi_base_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8152 entries, 0 to 8151
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                8152 non-null   int64         
 1   Grp.              8152 non-null   int64         
 2   Lact.             8152 non-null   int64         
 3   Parity            8152 non-null   int64         
 4   dim               8152 non-null   int64         
 5   Lactstage         8152 non-null   object        
 6   Avg.Conductivity  8152 non-null   object        
 7   DailyYield        8152 non-null   object        
 8   Daily avge yield  7738 non-null   object        
 9   Daily yield (%)   7738 non-null   object        
 10  day               8152 non-null   int64         
 11  Trt               8152 non-null   object        
 12  Batch             8152 non-null   int64         
 13  Bacteria          8152 non-null   object        
 14  Gram              8152 n

In [57]:
# Select only the required columns
# Using .copy() is a best practice to avoid 'SettingWithCopy' warnings later
afi_base_data_final = afi_base_data[['ID', 'DailyYield', 'Avg.Conductivity', 'Bacteria', 'Gram', 'Type', 'date']].copy()

# View the first few rows to confirm
print(afi_base_data_final.head())

     ID DailyYield Avg.Conductivity Bacteria Gram Type       date
0   893  80.080617              9.5       SU    P    E 2023-12-07
1   919   66.31497              9.9      MBG    P    C 2023-12-07
2   971          0                0       SC    P    E 2023-12-07
3  1643  47.511766             10.2      MBG    P    C 2023-12-07
4  1668  101.61314               10       SA    P    C 2023-12-07


In [58]:
import pandas as pd
from datetime import timedelta

def expand_afi_dates(df):
    # 1. Ensure date is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    expanded_list = []
    
    # 2. Process each cow (ID) individually
    for cow_id, group in df.groupby('ID'):
        # Find the original timeframe for this cow
        min_date = group['date'].min()
        max_date = group['date'].max()
        
        # Get the 'Baseline' values (values at the min and max dates)
        # We use .iloc[0] in case there are multiple entries on that day
        min_row = group[group['date'] == min_date].iloc[0]
        max_row = group[group['date'] == max_date].iloc[0]
        
        # 3. Create the new date range (30 days before min to 30 days after max)
        new_start = min_date - timedelta(days=30)
        new_end = max_end = max_date + timedelta(days=30)
        full_date_range = pd.date_range(new_start, new_end, freq='D')
        
        # 4. Create a "Template" dataframe for this ID with all dates
        temp_df = pd.DataFrame({'date': full_date_range})
        temp_df['ID'] = cow_id
        
        # 5. Merge existing data into this template
        # This keeps original Yield/Conductivity values on the days they existed
        merged = pd.merge(temp_df, group, on=['ID', 'date'], how='left')
        
        # 6. Apply the filling logic for Bacteria and Gram
        
        # Fill BEFORE minimum date
        before_mask = merged['date'] <= min_date
        merged.loc[before_mask, 'Bacteria'] = merged.loc[before_mask, 'Bacteria'].fillna(min_row['Bacteria'])
        merged.loc[before_mask, 'Gram'] = merged.loc[before_mask, 'Gram'].fillna(min_row['Gram'])
        merged.loc[before_mask, 'Type'] = merged.loc[before_mask, 'Type'].fillna(min_row['Type'])
        
        # Fill AFTER maximum date
        after_mask = merged['date'] >= max_date
        merged.loc[after_mask, 'Bacteria'] = merged.loc[after_mask, 'Bacteria'].fillna(max_row['Bacteria'])
        merged.loc[after_mask, 'Gram'] = merged.loc[after_mask, 'Gram'].fillna(max_row['Gram'])
        merged.loc[after_mask, 'Type'] = merged.loc[after_mask, 'Type'].fillna(max_row['Type'])
        
        expanded_list.append(merged)

    # Combine everything back together
    final_expanded_df = pd.concat(expanded_list, ignore_index=True)
    return final_expanded_df

# Run the function
afi_base_data_expanded = expand_afi_dates(afi_base_data_final)

In [59]:
# View the results for one ID to verify
afi_base_data_expanded.head()

,date,ID,DailyYield,Avg.Conductivity,Bacteria,Gram,Type
0,2023-11-29,445,NaN,NaN,Staph,P,E
1,2023-11-30,445,NaN,NaN,Staph,P,E
2,2023-12-01,445,NaN,NaN,Staph,P,E
3,2023-12-02,445,NaN,NaN,Staph,P,E
4,2023-12-03,445,NaN,NaN,Staph,P,E


In [12]:
posthmus_data = pd.read_excel('AFI data/Posthmus dairy daily milk data_AFI_20200101-20231231_beforedata.xlsx')

In [13]:
posthmus_data.head()

,Animal_ID,Group_ID,Date,Days_in_Milk,Age_Days,Lactation_Num,Yield(gr),Conductivity,Milking_Time(seconds)
0,421,2,2020-01-01,85,3375,8,0.0,0.0,0.0
1,477,5,2020-01-01,225,3737,7,0.0,0.0,0.0
2,1,11,2020-01-01,85,1614,1,0.0,0.0,0.0
3,499,2,2020-01-01,82,4383,9,0.0,0.0,0.0
4,50,10,2020-01-01,408,4029,8,0.0,0.0,0.0


In [14]:
len(posthmus_data['Animal_ID'].unique())

2759

In [16]:
posthmus_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412378 entries, 0 to 412377
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Animal_ID              412378 non-null  int64         
 1   Group_ID               412378 non-null  int64         
 2   Date                   412378 non-null  datetime64[ns]
 3   Days_in_Milk           412378 non-null  int64         
 4   Age_Days               412378 non-null  int64         
 5   Lactation_Num          412378 non-null  int64         
 6   Yield(gr)              411835 non-null  float64       
 7   Conductivity           411835 non-null  float64       
 8   Milking_Time(seconds)  411835 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(5)
memory usage: 28.3 MB


In [17]:
required_ids = afi_base_data['ID'].unique()

In [68]:
import pandas as pd
import glob
import os
import re

def concatenate_with_date(parent_folder="AFI data", sub_folder="OneDrive_AFI_after data"):
    path = os.path.join(parent_folder, sub_folder, "*.csv")
    all_files = glob.glob(path)
    
    df_list = []
    for filename in all_files:
        df = pd.read_csv(filename, low_memory=False)
        
        # --- EXTRACT DATE FROM FILENAME ---
        # This looks for a pattern like YYYY-MM-DD or DD-MM-YYYY in the filename
        base_name = os.path.basename(filename)
        
        # Try to find a date pattern (e.g., 2023-12-08 or 08-12-2023)
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})|(\d{2}-\d{2}-\d{4})', base_name)
        
        if date_match:
            df['date'] = date_match.group(0)
        else:
            # If the filename is just "20231208.csv", we can try this:
            # df['date'] = base_name.replace('.csv', '')
            df['date'] = "Unknown" # Fallback
            
        df_list.append(df)
    
    combined = pd.concat(df_list, ignore_index=True)
    combined['date'] = pd.to_datetime(combined['date'], errors='coerce')
    return combined

# Run this to get a version of afi_after_data that ACTUALLY has dates
afi_after_data = concatenate_with_date()

In [69]:
print(afi_after_data.head())

   Index    Cow  Grp.      Gyn. Lact.    DIM Cond. Cond..1 Cond..2   Avg.  \
0    NaN    NaN   NaN    status   no.    NaN  s. 1    s. 2    s. 3  cond.   
1    NaN    NaN   NaN       NaN   NaN    NaN   NaN     NaN     NaN    NaN   
2    1.0  441.0   5.0  Pregnant     3  110.0    --     8.2     7.9    8.2   
3    2.0  444.0   5.0      Bred     3   98.0    --     8.5     8.5    8.2   
4    3.0  445.0   1.0      Bred     4  177.0    --    10.0     9.5   10.3   

  Weight  Avg..1  Daily    Daily.1 Daily.2 Breed.   After Flow rate  \
0    NaN  weight  Yield  avg.yield   yield    no.  breed.      s. 1   
1    NaN     NaN    NaN        NaN     <%>    NaN     NaN       NaN   
2   1138    1151   38.4       59.1   -34.9      1      59        --   
3   1202      --   41.4       57.5   -28.0      2       9        --   
4   1576    1512   48.8       69.2   -29.5      2      31        --   

  Flow rate.1 Flow rate.2  Unnamed: 20       date  
0        s. 2        s. 3          NaN 2024-01-15  
1     

In [70]:
afi_after_data.head()

,Index,Cow,Grp.,Gyn.,Lact.,DIM,Cond.,Cond..1,Cond..2,Avg.,Weight,Avg..1,Daily,Daily.1,Daily.2,Breed.,After,Flow rate,Flow rate.1,Flow rate.2,Unnamed: 20,date
0,NaN,NaN,NaN,status,no.,NaN,s. 1,s. 2,s. 3,cond.,NaN,weight,Yield,avg.yield,yield,no.,breed.,s. 1,s. 2,s. 3,NaN,2024-01-15
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<%>,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-15
2,1.0,441.0,5.0,Pregnant,3,110.0,--,8.2,7.9,8.2,1138,1151,38.4,59.1,-34.9,1,59,--,3.5,3.6,NaN,2024-01-15
3,2.0,444.0,5.0,Bred,3,98.0,--,8.5,8.5,8.2,1202,--,41.4,57.5,-28.0,2,9,--,4.3,4.5,NaN,2024-01-15
4,3.0,445.0,1.0,Bred,4,177.0,--,10.0,9.5,10.3,1576,1512,48.8,69.2,-29.5,2,31,--,6.0,6.3,NaN,2024-01-15


In [72]:
afi_after_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141853 entries, 0 to 141852
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Index        141701 non-null  float64       
 1   Cow          141701 non-null  float64       
 2   Grp.         141701 non-null  float64       
 3   Gyn.         141777 non-null  object        
 4   Lact.        141777 non-null  object        
 5   DIM          141701 non-null  float64       
 6   Cond.        141777 non-null  object        
 7   Cond..1      141777 non-null  object        
 8   Cond..2      141777 non-null  object        
 9   Avg.         141777 non-null  object        
 10  Weight       141701 non-null  object        
 11  Avg..1       141777 non-null  object        
 12  Daily        141777 non-null  object        
 13  Daily.1      141777 non-null  object        
 14  Daily.2      141853 non-null  object        
 15  Breed.       141777 non-null  obje

In [22]:
events_data = pd.read_csv('events_23_24.csv')

In [23]:
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16900 entries, 0 to 16899
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           16900 non-null  int64  
 1   PEN          16900 non-null  int64  
 2   LACT         16900 non-null  int64  
 3   Event        16900 non-null  object 
 4   DIM          16900 non-null  int64  
 5   Date         16900 non-null  object 
 6   Remark       16900 non-null  object 
 7   R            16900 non-null  object 
 8   T            16900 non-null  object 
 9   B            16900 non-null  object 
 10  Protocols    16900 non-null  object 
 11  Technician   5276 non-null   object 
 12  Unnamed: 12  0 non-null      float64
dtypes: float64(1), int64(4), object(8)
memory usage: 1.7+ MB


In [25]:
events_data.head()

,ID,PEN,LACT,Event,DIM,Date,Remark,R,T,B,Protocols,Technician,Unnamed: 12
0,441,11,3,FRESH,0,09/27/23,,,,,,NaN,NaN
1,441,3,3,LAB,0,09/27/23,REPLI,,,,,NaN,NaN
2,441,3,3,LAB,0,09/27/23,PASTE,,,,,NaN,NaN
3,441,3,3,LAB,5,10/02/23,ERE,,,,,NaN,NaN
4,441,3,3,MOVE,43,11/09/23,F003T001,,,,,NaN,NaN


In [26]:
required_ids = afi_base_data['ID'].unique()

In [27]:
import pandas as pd
import glob
import os

def concatenate_afi_data(required_ids, parent_folder="rumination and water intake", sub_folder=""):
    # 1. Construct the full path
    folder_path = os.path.join(parent_folder, sub_folder)
    file_pattern = os.path.join(folder_path, "*.csv")
    all_files = glob.glob(file_pattern)

    if not all_files:
        print(f"No CSV files found in: {folder_path}")
        return None

    # 2. Convert IDs to strings (just in case they are numbers)
    # This ensures we can match them against the filename text
    id_set = set(str(id_val) for id_val in required_ids)

    # 3. Filter the file list
    # We only keep the file if one of our IDs is found in the filename
    filtered_files = []
    for f in all_files:
        filename = os.path.basename(f)
        # Check if any ID from our set exists within the current filename
        if any(id_str in filename for id_str in id_set):
            filtered_files.append(f)

    print(f"Found {len(all_files)} total files. Matched {len(filtered_files)} files with required IDs.")

    if not filtered_files:
        print("No files matched the required IDs.")
        return None

    # 4. Read only the matching files
    df_list = [pd.read_csv(filename, low_memory=False) for filename in filtered_files]

    # 5. Concatenate
    combined_df = pd.concat(df_list, ignore_index=True)

    print("Concatenation complete!")
    return combined_df

# --- How to use it ---
# Get your unique IDs from the base data
required_ids = afi_base_data['ID'].unique()

In [28]:
# Pass them into the function
rumination_and_water_intake_data = concatenate_afi_data(required_ids)

Found 2630 total files. Matched 249 files with required IDs.


C:\Users\Aditya Rajiv\AppData\Local\Temp\ipykernel_21228\4241472462.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(df_list, ignore_index=True)


Concatenation complete!


In [29]:
rumination_and_water_intake_data.head()

,Timestamp,temp_degree_celsius,animal_id,location,race,act_act,act_index_percent,drink_cycles_v2_number,temp_inc_index_number,temp_dec_index_number,temp_without_drink_cycles_degree_celsius,heat_index_percent,rum_index_seconds,water_intake_liter
0,2021-10-20 17:50:00,NaN,1012,4,JERSEY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-20 18:00:00,38.02,1012,4,JERSEY,6.12,14.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-20 18:10:00,39.69,1012,4,JERSEY,2.94,13.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-20 18:20:00,39.97,1012,4,JERSEY,4.17,12.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-10-20 18:30:00,40.02,1012,4,JERSEY,4.27,11.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
rumination_and_water_intake_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28790299 entries, 0 to 28790298
Data columns (total 14 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Timestamp                                 object 
 1   temp_degree_celsius                       float64
 2   animal_id                                 object 
 3   location                                  object 
 4   race                                      object 
 5   act_act                                   float64
 6   act_index_percent                         float64
 7   drink_cycles_v2_number                    float64
 8   temp_inc_index_number                     float64
 9   temp_dec_index_number                     float64
 10  temp_without_drink_cycles_degree_celsius  float64
 11  heat_index_percent                        float64
 12  rum_index_seconds                         float64
 13  water_intake_liter                        float64
dtype

In [32]:
# Filters rows where the timestamp string contains '00:00'
filtered_data = rumination_and_water_intake_data[
    rumination_and_water_intake_data['Timestamp'].str.contains('00:00', na=False)
]
print(f"Filtered down to {len(filtered_data)} rows.")

Filtered down to 4794819 rows.


In [41]:
rumination_and_water_intake_data_filtered = filtered_data.copy()

In [42]:
rumination_and_water_intake_data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4794819 entries, 1 to 28790297
Data columns (total 14 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Timestamp                                 object 
 1   temp_degree_celsius                       float64
 2   animal_id                                 object 
 3   location                                  object 
 4   race                                      object 
 5   act_act                                   float64
 6   act_index_percent                         float64
 7   drink_cycles_v2_number                    float64
 8   temp_inc_index_number                     float64
 9   temp_dec_index_number                     float64
 10  temp_without_drink_cycles_degree_celsius  float64
 11  heat_index_percent                        float64
 12  rum_index_seconds                         float64
 13  water_intake_liter                        float64
dtypes: flo

In [43]:
rumination_and_water_intake_data_filtered.head()

,Timestamp,temp_degree_celsius,animal_id,location,race,act_act,act_index_percent,drink_cycles_v2_number,temp_inc_index_number,temp_dec_index_number,temp_without_drink_cycles_degree_celsius,heat_index_percent,rum_index_seconds,water_intake_liter
1,2021-10-20 18:00:00,38.02,1012,4,JERSEY,6.12,14.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-10-20 19:00:00,40.02,1012,4,JERSEY,3.27,10.09,NaN,NaN,NaN,40.05,NaN,NaN,NaN
13,2021-10-20 20:00:00,38.04,1012,4,JERSEY,4.38,8.28,NaN,NaN,NaN,39.92,NaN,NaN,NaN
19,2021-10-20 21:00:00,39.03,1012,4,JERSEY,2.94,12.11,NaN,NaN,NaN,39.62,NaN,NaN,NaN
25,2021-10-20 22:00:00,39.58,1012,4,JERSEY,1.56,4.14,NaN,NaN,NaN,39.51,NaN,NaN,NaN


In [44]:
rumination_and_water_final = rumination_and_water_intake_data_filtered[['Timestamp', 'animal_id', 'rum_index_seconds', 'water_intake_liter']]

In [38]:
rumination_and_water_final

,Timestamp,rum_index_seconds,water_intake_liter
1,2021-10-20 18:00:00,NaN,NaN
7,2021-10-20 19:00:00,NaN,NaN
13,2021-10-20 20:00:00,NaN,NaN
19,2021-10-20 21:00:00,NaN,NaN
25,2021-10-20 22:00:00,NaN,NaN
...,...,...,...
28790273,2025-04-04 07:00:00,34042.0,0.0
28790279,2025-04-04 08:00:00,34042.0,0.0
28790285,2025-04-04 09:00:00,35979.0,0.0
28790291,2025-04-04 10:00:00,35182.0,0.0


In [45]:
# 1. Convert the 'Timestamp' column to a proper datetime object first
# (This ensures Pandas recognizes the text as a date/time)
rumination_and_water_final['Timestamp'] = pd.to_datetime(rumination_and_water_final['Timestamp'])

# 2. Extract just the date component
# This will create a new column 'date' with format: 2021-10-20
rumination_and_water_final['date'] = rumination_and_water_final['Timestamp'].dt.date

# Optional: To see the result
print(rumination_and_water_final[['Timestamp', 'date']].head())

             Timestamp        date
1  2021-10-20 18:00:00  2021-10-20
7  2021-10-20 19:00:00  2021-10-20
13 2021-10-20 20:00:00  2021-10-20
19 2021-10-20 21:00:00  2021-10-20
25 2021-10-20 22:00:00  2021-10-20


C:\Users\Aditya Rajiv\AppData\Local\Temp\ipykernel_21228\106079818.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rumination_and_water_final['Timestamp'] = pd.to_datetime(rumination_and_water_final['Timestamp'])
C:\Users\Aditya Rajiv\AppData\Local\Temp\ipykernel_21228\106079818.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rumination_and_water_final['date'] = rumination_and_water_final['Timestamp'].dt.date


In [46]:
rumination_and_water_final

,Timestamp,animal_id,rum_index_seconds,water_intake_liter,date
1,2021-10-20 18:00:00,1012,NaN,NaN,2021-10-20
7,2021-10-20 19:00:00,1012,NaN,NaN,2021-10-20
13,2021-10-20 20:00:00,1012,NaN,NaN,2021-10-20
19,2021-10-20 21:00:00,1012,NaN,NaN,2021-10-20
25,2021-10-20 22:00:00,1012,NaN,NaN,2021-10-20
...,...,...,...,...,...
28790273,2025-04-04 07:00:00,9800,34042.0,0.0,2025-04-04
28790279,2025-04-04 08:00:00,9800,34042.0,0.0,2025-04-04
28790285,2025-04-04 09:00:00,9800,35979.0,0.0,2025-04-04
28790291,2025-04-04 10:00:00,9800,35182.0,0.0,2025-04-04


In [48]:
rumination_and_water_final = rumination_and_water_final.drop(['Timestamp'], axis = 1)

In [49]:
rumination_and_water_final

,animal_id,rum_index_seconds,water_intake_liter,date
1,1012,NaN,NaN,2021-10-20
7,1012,NaN,NaN,2021-10-20
13,1012,NaN,NaN,2021-10-20
19,1012,NaN,NaN,2021-10-20
25,1012,NaN,NaN,2021-10-20
...,...,...,...,...
28790273,9800,34042.0,0.0,2025-04-04
28790279,9800,34042.0,0.0,2025-04-04
28790285,9800,35979.0,0.0,2025-04-04
28790291,9800,35182.0,0.0,2025-04-04


In [ ]:
# rumination_and_water_final
# events_data
# afi_after_data
# afi_base_data_final
# posthmus_data

In [ ]:
# afi_after_data
# afi_base_data_final
# posthmus_data

In [60]:
afi_after_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141853 entries, 0 to 141852
Data columns (total 21 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Index        141701 non-null  float64
 1   Cow          141701 non-null  float64
 2   Grp.         141701 non-null  float64
 3   Gyn.         141777 non-null  object 
 4   Lact.        141777 non-null  object 
 5   DIM          141701 non-null  float64
 6   Cond.        141777 non-null  object 
 7   Cond..1      141777 non-null  object 
 8   Cond..2      141777 non-null  object 
 9   Avg.         141777 non-null  object 
 10  Weight       141701 non-null  object 
 11  Avg..1       141777 non-null  object 
 12  Daily        141777 non-null  object 
 13  Daily.1      141777 non-null  object 
 14  Daily.2      141853 non-null  object 
 15  Breed.       141777 non-null  object 
 16  After        141777 non-null  object 
 17  Flow rate    141777 non-null  object 
 18  Flow rate.1  141777 non-

In [61]:
afi_base_data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8152 entries, 0 to 8151
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                8152 non-null   int64         
 1   DailyYield        8152 non-null   object        
 2   Avg.Conductivity  8152 non-null   object        
 3   Bacteria          8152 non-null   object        
 4   Gram              8152 non-null   object        
 5   Type              8152 non-null   object        
 6   date              8152 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 445.9+ KB


In [62]:
posthmus_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412378 entries, 0 to 412377
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Animal_ID              412378 non-null  int64         
 1   Group_ID               412378 non-null  int64         
 2   Date                   412378 non-null  datetime64[ns]
 3   Days_in_Milk           412378 non-null  int64         
 4   Age_Days               412378 non-null  int64         
 5   Lactation_Num          412378 non-null  int64         
 6   Yield(gr)              411835 non-null  float64       
 7   Conductivity           411835 non-null  float64       
 8   Milking_Time(seconds)  411835 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(5)
memory usage: 28.3 MB


In [73]:
import pandas as pd
import numpy as np

# 1. Prepare posthmus_data (The "Before" source)
post_subset = posthmus_data[['Animal_ID', 'Date', 'Yield(gr)', 'Conductivity']].copy()
post_subset.columns = ['ID', 'date', 'yield_post', 'cond_post']

# 2. Prepare afi_after_data (The "After" source)
# We assume 'Cow' is the ID. Note: Ensure this has a 'date' column!
# If 'Daily' is your yield and 'Avg.' is your conductivity:
after_subset = afi_after_data[['Cow', 'date', 'Daily', 'Avg.']].copy() 
after_subset.columns = ['ID', 'date', 'yield_after', 'cond_after']

# 3. Merge both into your master skeleton
final_merge = pd.merge(afi_base_data_final, post_subset, on=['ID', 'date'], how='left')
final_merge = pd.merge(final_merge, after_subset, on=['ID', 'date'], how='left')

# 4. Convert all yield/cond columns to numeric (crucial for filling)
cols_to_fix = ['DailyYield', 'Avg.Conductivity', 'yield_post', 'cond_post', 'yield_after', 'cond_after']
for col in cols_to_fix:
    final_merge[col] = pd.to_numeric(final_merge[col], errors='coerce')

# 5. The Logic: Fill missing values based on the source
# We use .fillna() to prioritize the original data, then the post/after data
final_merge['DailyYield'] = final_merge['DailyYield'].fillna(final_merge['yield_post']).fillna(final_merge['yield_after'])
final_merge['Avg.Conductivity'] = final_merge['Avg.Conductivity'].fillna(final_merge['cond_post']).fillna(final_merge['cond_after'])

# 6. Final Clean up: Select only the requested columns and rename to your final names
final_dataset = final_merge.rename(columns={
    'DailyYield': 'yield',
    'Avg.Conductivity': 'conductivity',
    'Bacteria': 'bacteria',
    'Gram': 'gram',
    'Type': 'type'
})[['ID', 'date', 'yield', 'conductivity', 'bacteria', 'gram', 'type']]

print("Data successfully patched and joined!")

Data successfully patched and joined!


In [74]:
final_dataset

,ID,date,yield,conductivity,bacteria,gram,type
0,893,2023-12-07,80.080617,9.5,SU,P,E
1,919,2023-12-07,66.314970,9.9,MBG,P,C
2,971,2023-12-07,0.000000,0.0,SC,P,E
3,1643,2023-12-07,47.511766,10.2,MBG,P,C
4,1668,2023-12-07,101.613140,10.0,SA,P,C
...,...,...,...,...,...,...,...
8147,9565,2024-02-29,51.700000,8.8,Coryne,P,E
8148,9575,2024-02-29,53.900000,8.2,MBG,P,C
8149,50068,2024-02-29,44.300000,9.7,CON,P,C
8150,50130,2024-02-29,32.000000,8.2,MBG,P,C


In [77]:
rumination_and_water_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4794819 entries, 1 to 28790297
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   animal_id           object 
 1   rum_index_seconds   float64
 2   water_intake_liter  float64
 3   date                object 
dtypes: float64(2), object(2)
memory usage: 311.9+ MB


In [76]:
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16900 entries, 0 to 16899
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           16900 non-null  int64  
 1   PEN          16900 non-null  int64  
 2   LACT         16900 non-null  int64  
 3   Event        16900 non-null  object 
 4   DIM          16900 non-null  int64  
 5   Date         16900 non-null  object 
 6   Remark       16900 non-null  object 
 7   R            16900 non-null  object 
 8   T            16900 non-null  object 
 9   B            16900 non-null  object 
 10  Protocols    16900 non-null  object 
 11  Technician   5276 non-null   object 
 12  Unnamed: 12  0 non-null      float64
dtypes: float64(1), int64(4), object(8)
memory usage: 1.7+ MB


In [81]:
import pandas as pd

# ---------------------------------------------------------
# 1. PREPARE RUMINATION & WATER DATA
# ---------------------------------------------------------
# Standardize ID and Date, then aggregate to one row per day
rum_water_prep = rumination_and_water_final.copy()
rum_water_prep['ID'] = pd.to_numeric(rum_water_prep['animal_id'], errors='coerce')
rum_water_prep['date'] = pd.to_datetime(rum_water_prep['date'])

rum_water_daily = rum_water_prep.groupby(['ID', 'date']).agg({
    'rum_index_seconds': 'mean',
    'water_intake_liter': 'sum'
}).reset_index()

# ---------------------------------------------------------
# 2. PREPARE EVENTS DATA (CLEANED & DE-DUPLICATED)
# ---------------------------------------------------------
events_prep = events_data.copy()
events_prep['date'] = pd.to_datetime(events_prep['Date'])
events_prep['ID'] = pd.to_numeric(events_prep['ID'], errors='coerce')

def clean_event_list(event_series):
    """Strips whitespace, removes duplicates, and returns a sorted list."""
    # 1. str(s).strip() removes the "LAB        " spaces
    # 2. set() removes duplicates within the same day
    # 3. sorted() keeps the order consistent (e.g., ['FRESH', 'LAB'])
    cleaned = sorted(list(set(str(s).strip() for s in event_series if pd.notna(s))))
    return cleaned

events_daily = events_prep.groupby(['ID', 'date'])['Event'].apply(clean_event_list).reset_index()

# ---------------------------------------------------------
# 3. FINAL MASTER JOIN
# ---------------------------------------------------------
# Ensure our skeleton (final_dataset) matches the formats
final_dataset['ID'] = pd.to_numeric(final_dataset['ID'], errors='coerce')
final_dataset['date'] = pd.to_datetime(final_dataset['date'])

# Step A: Merge Rumination/Water Intake
master_df = pd.merge(final_dataset, rum_water_daily, on=['ID', 'date'], how='left')

# Step B: Merge Cleaned Events List
master_df = pd.merge(master_df, events_daily, on=['ID', 'date'], how='left')

# Step C: Post-Merge Cleanup
# Replace NaN in 'Event' with an empty list [] for a cleaner dataset
master_df['Event'] = master_df['Event'].apply(lambda d: d if isinstance(d, list) else [])

print("Master dataset complete!")
print(f"Total records: {len(master_df)}")
print(master_df.head())

C:\Users\Aditya Rajiv\AppData\Local\Temp\ipykernel_21228\3299171587.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events_prep['date'] = pd.to_datetime(events_prep['Date'])


Master dataset complete!
Total records: 8152
     ID       date       yield  conductivity bacteria gram type  \
0   893 2023-12-07   80.080617           9.5       SU    P    E   
1   919 2023-12-07   66.314970           9.9      MBG    P    C   
2   971 2023-12-07    0.000000           0.0       SC    P    E   
3  1643 2023-12-07   47.511766          10.2      MBG    P    C   
4  1668 2023-12-07  101.613140          10.0       SA    P    C   

   rum_index_seconds  water_intake_liter Event  
0       27688.625000               17.38    []  
1       28818.625000                0.00    []  
2       22532.125000                0.00    []  
3       29386.500000                0.00    []  
4       31987.083333               15.01    []  


In [79]:
master_df

,ID,date,yield,conductivity,bacteria,gram,type,rum_index_seconds,water_intake_liter,Event
0,893,2023-12-07,80.080617,9.5,SU,P,E,27688.625000,17.38,[]
1,919,2023-12-07,66.314970,9.9,MBG,P,C,28818.625000,0.00,[]
2,971,2023-12-07,0.000000,0.0,SC,P,E,22532.125000,0.00,[]
3,1643,2023-12-07,47.511766,10.2,MBG,P,C,29386.500000,0.00,[]
4,1668,2023-12-07,101.613140,10.0,SA,P,C,31987.083333,15.01,[]
...,...,...,...,...,...,...,...,...,...,...
8147,9565,2024-02-29,51.700000,8.8,Coryne,P,E,24284.500000,21.49,[]
8148,9575,2024-02-29,53.900000,8.2,MBG,P,C,24860.958333,38.97,[]
8149,50068,2024-02-29,44.300000,9.7,CON,P,C,29237.833333,23.01,[]
8150,50130,2024-02-29,32.000000,8.2,MBG,P,C,25239.291667,8.77,[]


In [83]:
print(len(final_dataset)) 
# If this says ~8,000, then the 30-day expansion wasn't saved to this variable.

8152


In [84]:
master_df.to_csv('final_dataset.csv', index = False)